In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score, KFold

In [33]:
#Import Datasets, remove All Star Team from the main Dataframe, and remove the "Tm" label from both datasets
df_main = pd.read_csv("./Data/cleaned_data.csv", delimiter = '\t')
df_all_stars = pd.read_csv("./Data/all_stars.csv", delimiter = '\t')
df_main = df_main.drop(columns =['Tm'])
df_all_stars = df_all_stars.drop(columns =['Tm'])
df_main

,Player,PTS,Age,G,GS,MP,FG,FGA,FG%,3P,...,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF
0,Precious Achiuwa,5.2,-0.635196,0.206292,-0.315059,0.032834,-0.265682,-0.331798,0.435515,-0.957010,...,-0.136166,-0.539862,1.715319,-0.366901,0.246799,-0.605531,-0.277504,1.987459,-0.591965,0.521759
1,Bam Adebayo,22.6,-0.195287,-0.526429,0.200930,1.376889,2.154425,2.038275,0.473375,-0.541172,...,1.154829,0.363192,1.496885,1.984685,1.998319,0.998600,-0.277504,-0.830641,0.663333,1.215277
2,Nickeil Alexander-Walker,7.3,-0.415242,1.488553,-0.487055,0.271777,-0.166225,0.025336,-0.224329,0.602382,...,-0.505022,1.148218,-0.578238,-0.541093,-0.561594,0.246663,0.123752,0.036467,-0.068924,-0.085069
3,Grayson Allen,3.5,0.244623,-1.075970,-0.315059,0.114970,-0.862421,-0.705166,-1.122148,-0.437213,...,0.140475,1.148218,0.186281,0.155674,0.179433,-0.405015,-0.076876,-0.830641,0.558725,0.348380
4,Jarrett Allen,17.0,-0.415242,-0.709609,0.028934,0.884069,1.060404,0.398704,1.452323,-0.957010,...,2.215289,0.302805,3.244357,3.204025,3.480375,-0.254628,1.528145,1.337129,0.349509,0.348380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,Jaylin Williams,4.4,-1.295060,0.389472,-0.659052,-0.542123,-0.497747,-0.575299,0.419289,-0.021375,...,-0.597236,0.462006,-0.469021,0.068578,-0.090031,-0.154370,-0.277504,-0.180310,-0.696573,-0.431829
210,Kenrich Williams,0.6,0.464577,-0.160069,-0.659052,-1.146948,-0.928725,-0.932433,-0.851721,-0.957010,...,-0.781664,-1.596628,-0.796672,-0.715284,-0.797376,-0.705790,-0.879387,-0.830641,-1.010398,-1.038657
211,Delon Wright,8.0,0.904487,-0.709609,-0.487055,0.510720,-0.033617,-0.299331,1.041273,0.602382,...,-0.551129,1.148218,-0.359804,-0.062066,-0.157398,-0.003982,0.926262,-0.180310,-1.010398,-0.258449
212,Thaddeus Young,0.0,1.784306,-1.259150,-0.659052,-1.191750,-1.028181,-1.111000,-2.203859,-0.957010,...,-0.781664,-1.596628,-0.905889,-1.150763,-1.167890,-0.906306,-1.080014,-0.830641,-1.010398,-1.385416


In [34]:
df_all_stars

,Player,PTS,Age,G,GS,MP,FG,FGA,FG%,3P,...,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF
0,Paolo Banchero,27.0,-1.295060,-0.160069,0.544923,1.302219,2.187577,2.346709,0.262441,1.434057,...,2.445823,0.475731,1.278451,1.723397,1.728855,1.098858,1.126890,0.470021,3.801580,0.348380
1,Devin Booker,27.5,0.024668,-0.709609,0.028934,1.608365,1.623991,1.535040,0.457149,0.914260,...,3.967353,1.013720,-0.578238,0.155674,-0.056348,2.101440,2.531283,-0.180310,1.918632,2.515624
2,Jalen Brunson,32.4,0.024668,0.939012,1.576901,1.481427,2.817468,3.142145,0.197539,1.122179,...,3.460177,0.530627,-0.250587,0.025030,-0.056348,2.853377,0.525007,-0.397087,1.814024,0.348380
3,Luka Don?i?,28.9,-0.635196,2.587634,3.124869,1.563563,2.353338,2.606443,0.208356,2.265732,...,2.445823,0.503179,-0.141370,2.637903,2.032002,3.154151,2.731911,0.036467,3.278539,0.781829
4,Kevin Durant,26.8,1.784306,-0.709609,0.028934,1.645700,2.054969,1.616207,0.781662,0.394463,...,3.137428,0.665125,-0.578238,1.592754,1.021510,0.747954,-0.076876,2.421013,1.604807,0.348380
5,Anthony Edwards,27.6,-1.075106,1.488553,2.092891,1.541162,2.220729,2.184375,0.397655,2.057814,...,2.169182,0.637676,0.295498,1.418562,1.189925,2.352086,1.929400,0.470021,2.441673,0.608449
6,Joel Embiid,33.0,0.464577,-0.343249,0.372927,1.593431,2.220729,2.492810,0.197539,1.330098,...,5.212241,0.761194,2.589055,2.202424,2.469882,1.951053,1.327517,2.421013,3.383147,1.475347
7,Shai Gilgeous-Alexander,30.2,-0.415242,0.389472,1.060912,1.488894,2.651707,2.525276,0.478783,0.706341,...,2.953000,0.571800,-0.032153,1.636302,1.257292,2.301957,1.528145,2.854567,1.290983,1.301967
8,LeBron James,27.8,2.664124,-0.526429,0.200930,1.556096,2.684859,2.103209,0.857382,1.122179,...,1.339256,0.431813,-0.032153,1.462110,1.122559,3.505055,3.735049,1.337129,2.964714,0.521759
9,Nikola Joki?,28.7,0.244623,0.755832,1.404905,1.511294,2.651707,2.184375,0.743802,0.290503,...,1.938647,0.876478,3.025923,3.116930,3.345642,3.454926,1.728773,0.686798,2.441673,0.955208


## Traditional Test/Train split doesnt work in this case. Solution: 
### Using a loop parameter (0 to 999) as the random_seed value in each iteration, grab 30 players, split the train df halfway, use discrete variable 'Winner' to decide which team wins (which team has a higher avg PTS). If both teams have the same avg PTS, then that iteration is skipped.

### Once fitting loop is done, run a smaller loop (20 iterations) and this time use confusion matrix, accuracy, f1, etc to evaluate the performances.

### For both of these, there will be a safeguard and check to see if any of the random teams happen to be the All Star Team. This has a very low chance of occuring, but that iteration would be skipped.

### Then use a loop to assemble more random teams to challenge the All Star Team and see how they do.

In [35]:
#Initializing Model Object
SGD = SGDClassifier(loss="log_loss")

#Training Loop
for i in range(1000):
    #Getting random sample, and splitting into two teams
    df_train = df_main.sample(n=30, random_state = i)
    df_train_A = df_train.iloc[:15, :].copy()
    df_train_B = df_train.iloc[15:, :].copy()
    
    #Checking to see either team happens to be the all star team
    if (set(df_all_stars['Player']) == set(df_train_A['Player']) or set(df_all_stars['Player']) == set(df_train_B['Player'])):
        print(f"All star team was randomly selected in iteration {i}. Skipping iteration")
        continue
    
    #Getting the average score, and comparing scores to assign a winning and losing team
    score_A = np.mean(df_train_A['PTS'])
    score_B = np.mean(df_train_B['PTS'])
    
    if score_A > score_B:
        df_train_A['Winner'] = 1
        df_train_B['Winner'] = 0
        
    elif score_A < score_B:
        df_train_A['Winner'] = 0
        df_train_B['Winner'] = 1      
    
    #In the event of a tie
    else:
        print(f"Theoretical tie in iteration {i}. Skipping iteration")
        continue
        
    #Dropping 'PTS' and 'Player' column from both dfs, and merging them back together
    df_train_A = df_train_A.drop(columns=['Player', 'PTS'])
    df_train_B = df_train_B.drop(columns=['Player', 'PTS'])
    df_train_final = pd.concat([df_train_A, df_train_B], axis = 0)
    X = df_train_final.iloc[:, :-1]
    y = df_train_final['Winner']
    
    SGD.partial_fit(X, y, classes= [0, 1])

Theoretical tie in iteration 169. Skipping iteration


In [ ]:
df_all_stars = df_all_stars.drop(columns=['PTS'])
df_all_stars['Winner'] = 1